In [231]:
import tensorflow as tf

In [232]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [233]:
tf.test.is_gpu_available()

True

In [234]:
!pip list

Package                   Version
------------------------- ----------
anyio                     4.1.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.13.1
beautifulsoup4            4.12.2
bleach                    6.1.0
blinker                   1.7.0
certifi                   2021.5.30
cffi                      1.16.0
chardet                   4.0.0
charset-normalizer        3.3.2
click                     8.1.7
colorama                  0.4.6
comm                      0.2.0
contourpy                 1.2.0
cycler                    0.12.1
debugpy                   1.8.0
decorator                 5.1.1
defusedxml                0.7.1
distlib                   0.3.7
executing                 2.0.1
fastjsonschema            2.19.0
filelock                  3.13.1
Flask                     3.0.2
flask-ngrok         


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\micha\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [235]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\micha\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [236]:

# Import library
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.tag import pos_tag
from nltk.corpus import stopwords
import json
import pickle
import numpy as np
from keras.models import Sequential
from keras.layers import Dense,Activation,Dropout
from keras.optimizers import SGD,Adam,RMSprop,Adagrad,Adadelta,Adamax,Nadam
import string
import random

# What is SGD? Stochastic Gradient Descent
# What is it for? To optimize the model
# Why use SGD? Because SGD is the most commonly used optimizer
# What are the advantages of SGD? Fast convergence speed
# What are the disadvantages of SGD? Unstable
# What is the solution to the disadvantages of SGD? Using a small learning rate
# What is a learning rate? The learning rate is a hyperparameter that determines how big a step is taken when optimizing the model
# What is a hyperparameter? Hyperparameters are parameters used to optimize the model
# What is a parameter? Parameters are variables whose values are changed by the model to optimize the model

In [237]:
# Initialize

lemmatizer=WordNetLemmatizer()
words=[]
classes=[]
documents=[]
punc=string.punctuation
stop_words=stopwords.words('english')

In [238]:
# Json file
# 1.
with open("./Dataset/intents.json",encoding="utf-8") as file:
    intents=json.load(file)

# 2.
# data_file=open("intents.json").read() # return string type
# intents=json.loads(data_file)
# print(type(intents))

In [239]:
intents["intents"][0]

{'tag': 'greeting',
 'patterns': ['Hi there',
  'How are you',
  'Is anyone there?',
  'Hey',
  'Hola',
  'Hello',
  'Good day'],
 'responses': ['Hello, thanks for asking',
  'Good to see you again',
  'Hi there, how can I help?'],
 'context': ['']}

In [240]:
# function to clean
def remove_punctuation(w_list):
    return [word for word in w_list if word not in punc]

def remove_stopwords(w_list):
    return [word for word in w_list if word not in stop_words]

def remove_number(w_list):
    return [word for word in w_list if not word.isdigit()]

def get_tag(tag):
    if tag.startswith('j'):
        return 'a'
    elif tag.startswith('v'):
        return 'v'
    elif tag.startswith('n'):
        return 'n'
    elif tag.startswith('r'):
        return 'r'
    else:
        return None

def lemmatize(w_list):
    lemmatized=[]
    tagging=pos_tag(w_list)
    for word,tag in tagging:
        tag=get_tag(tag.lower())
        if tag is None:
            lemmatized.append(word)
        else:
            lemmatized.append(lemmatizer.lemmatize(word,tag))
    return lemmatized

In [241]:
# test=word_tokenize("I am, a student")
# print(remove_punctuation(test))

In [242]:
# intents["intents"][0]

In [243]:
# Populating the lists
for intent in intents["intents"]:
    for pattern in intent["patterns"]:
        # print("pattern: ->",pattern)
        
        # tokenize it
        word_list=word_tokenize(pattern)
        word_list=[word.lower() for word in word_list]
        # word_list=remove_punctuation(word_list)
        # word_list=remove_stopwords(word_list)
        words.extend(word_list)
        
        # print(words)
        
        # add documents
        documents.append((word_list,intent["tag"].strip()))
        
        # add classes to class list
        if intent["tag"].strip() not in classes:
            classes.append(intent["tag"].strip())

words=remove_punctuation(words)
words=remove_stopwords(words)
# words=remove_number(words)
words=lemmatize(words)

# words=sorted(set(words))

classes=sorted(set(classes))
# print(words)
print(len(documents),"Documents:",documents)
print(len(classes),"Classes:",classes)
print(len(words),"Unique lemmatized words: ",words)

915 Documents: [(['hi', 'there'], 'greeting'), (['how', 'are', 'you'], 'greeting'), (['is', 'anyone', 'there', '?'], 'greeting'), (['hey'], 'greeting'), (['hola'], 'greeting'), (['hello'], 'greeting'), (['good', 'day'], 'greeting'), (['bye'], 'goodbye'), (['see', 'you', 'later'], 'goodbye'), (['goodbye'], 'goodbye'), (['nice', 'chatting', 'to', 'you', ',', 'bye'], 'goodbye'), (['till', 'next', 'time'], 'goodbye'), (['thanks'], 'thanks'), (['thank', 'you'], 'thanks'), (['that', "'s", 'helpful'], 'thanks'), (['awesome', ',', 'thanks'], 'thanks'), (['thanks', 'for', 'helping', 'me'], 'thanks'), (['how', 'you', 'could', 'help', 'me', '?'], 'options'), (['what', 'you', 'can', 'do', '?'], 'options'), (['what', 'help', 'you', 'provide', '?'], 'options'), (['how', 'you', 'can', 'be', 'helpful', '?'], 'options'), (['what', 'support', 'is', 'offered'], 'options'), (['how', 'to', 'check', 'adverse', 'drug', 'reaction', '?'], 'adverse_drug'), (['open', 'adverse', 'drugs', 'module'], 'adverse_drug'

In [244]:
for i,test in enumerate(classes):
    if test =="Support Vector Machine":
        print(f"{i} ketemu")

79 ketemu


In [245]:
with open("./Dataset/words.pkl","wb") as f:
    pickle.dump(words,f)
with open("./Dataset/classes.pkl","wb") as f:
    pickle.dump(classes,f)
with open("./Dataset/documents.pkl","wb") as f:
    pickle.dump(documents,f)

In [246]:
# Initialize training data

training=[]
output_empty=[0]*len(classes)
for doc in documents:
    
    # bag of word
    bag=[]
    
    # list of tokenized words for pattern
    pattern_words=doc[0]
    pattern_words=remove_punctuation(pattern_words)
    pattern_words=remove_stopwords(pattern_words)
    # pattern_words=remove_number(pattern_words)
    pattern_words=lemmatize(pattern_words)
    
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    output_row=output_empty.copy()
    # print(output_row)
    output_row[classes.index(doc[1])]=1
    # print(output_row)
    # print(output_row.index(1))
    # break
    
    # print(len(bag))
    # print(len(output_row))
    
    training.append([bag,output_row])

    # print(bag)
    # print(output_row)

    
    # print(training[0][0])
    # print(training[0][1])
    # break
    
# print(len(training))

# Shuffle features
random.shuffle(training)

# Cara 1
train_x=[item[0] for item in training]
train_y=[item[1] for item in training]

print(len(train_x))
print(len(train_y))


# Cara 2
# training_data=np.array(training)
# # print(training_data)

# train_x=list(training_data[:,0])
# train_y=list(training_data[:,1])

# print(train_x.shape)
# print(train_y.shape)

print("Training data created!")

# print("train_x:",train_x)
# print("train_y:",train_y)



915
915
Training data created!


In [247]:
# Training model

# adam=Adam(learning_rate=0.001)
# rmsprop=RMSprop(learning_rate=0.001)
# adagrad=Adagrad(learning_rate=0.01)
# nadam=Nadam(learning_rate=0.001,beta_1=-0.9,beta_2=0.999)

model=Sequential()
model.add(Dense(128,input_shape=(len(train_x[0]),),activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.2))
# model.add(Dense(32,activation="relu"))
# model.add(Dropout(0.2))
model.add(Dense(len(train_y[0]),activation='softmax'))


sgd=SGD(learning_rate=0.01,decay=1e-6,momentum=0.5,nesterov=True)
model.compile(loss="categorical_crossentropy",optimizer=sgd,metrics=["accuracy"])
hist=model.fit(np.array(train_x),np.array(train_y),epochs=200,batch_size=2,verbose=1)
model.save("./Model/Model_Chatbot_Personal.h5",hist)

print("Model Created!")

Epoch 1/200
458/458 [==============================] - 1s 2ms/step - loss: 5.5019 - accuracy: 0.0109
Epoch 2/200
458/458 [==============================] - 1s 2ms/step - loss: 5.1761 - accuracy: 0.0383
Epoch 3/200
458/458 [==============================] - 1s 2ms/step - loss: 4.6871 - accuracy: 0.0885
Epoch 4/200
458/458 [==============================] - 1s 2ms/step - loss: 4.1865 - accuracy: 0.1311
Epoch 5/200
458/458 [==============================] - 6s 13ms/step - loss: 3.7943 - accuracy: 0.1770
Epoch 6/200
458/458 [==============================] - 1s 2ms/step - loss: 3.4186 - accuracy: 0.2404
Epoch 7/200
458/458 [==============================] - 1s 2ms/step - loss: 3.0961 - accuracy: 0.2885
Epoch 8/200
458/458 [==============================] - 1s 2ms/step - loss: 2.7683 - accuracy: 0.3486
Epoch 9/200
458/458 [==============================] - 1s 2ms/step - loss: 2.5831 - accuracy: 0.3541
Epoch 10/200
458/458 [==============================] - 1s 2ms/step - loss: 2.2324 - accur

In [248]:
# from sklearn.ensemble import RandomForestClassifier

# # Initialize Random Forest classifier
# model = RandomForestClassifier(n_estimators=100)

# # Training the model
# model.fit(train_x, train_y)

# # Evaluating the model
# accuracy = model.score(train_x, train_y)
# print("Accuracy:", accuracy)

In [249]:
!pip list

Package                   Version
------------------------- ----------
anyio                     4.1.0
argon2-cffi               23.1.0
argon2-cffi-bindings      21.2.0
arrow                     1.3.0
asttokens                 2.4.1
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.13.1
beautifulsoup4            4.12.2
bleach                    6.1.0
blinker                   1.7.0
certifi                   2021.5.30
cffi                      1.16.0
chardet                   4.0.0
charset-normalizer        3.3.2
click                     8.1.7
colorama                  0.4.6
comm                      0.2.0
contourpy                 1.2.0
cycler                    0.12.1
debugpy                   1.8.0
decorator                 5.1.1
defusedxml                0.7.1
distlib                   0.3.7
executing                 2.0.1
fastjsonschema            2.19.0
filelock                  3.13.1
Flask                     3.0.2
flask-ngrok         


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: C:\Users\micha\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [250]:
test=[1,2,3,4,5,6,9]
test2=[0]*len(test)
test2[test.index(3)]=1
test2

[0, 0, 1, 0, 0, 0, 0]

In [251]:
print(lemmatizer.lemmatize("running",pos="v"))

run
